In [ ]:
import os
import numpy as np
import csv
import pandas as pd
import math
import time
import tensorflow as tf
import unicodedata
from sklearn.preprocessing import MinMaxScaler
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
apple_price = apple_data.drop(["News"], axis = 1)
apple_price.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2006-12-01,13.114285,13.190000,12.871428,91.320000,13.045714,198769900
1,2006-12-04,13.125714,13.150000,12.928572,91.120003,13.017143,177384200
2,2006-12-05,13.092857,13.190000,12.981428,91.269997,13.038571,165709600
3,2006-12-06,12.948571,13.055715,12.810000,89.830002,12.832857,159546100
4,2006-12-07,12.861428,12.928572,12.414286,87.040001,12.434286,251206900


In [ ]:
apple_price.to_csv(rpath + "Apple_Price.csv")

## Load text data

In [ ]:
rpath = "/content/drive/MyDrive/UofT/grad/ece1508/Final_Project/sentiment_analysis/"

In [ ]:
apple_data = pd.read_csv(rpath + 'AppleNewsStock.csv')

In [ ]:
apple_data['News'] = apple_data['News'].fillna("DNE")

In [ ]:
print(apple_data.shape)
apple_data.head()

(2517, 8)


,Date,Open,High,Low,Close,Adj Close,Volume,News
0,2006-12-01,13.114285,13.190000,12.871428,91.320000,13.045714,198769900,WHAT'S ON TONIGHT : 8 P.M. (TLC) ASHLEY JUDD A...
1,2006-12-04,13.125714,13.150000,12.928572,91.120003,13.017143,177384200,More on Housing Prices : The broadest governme...
2,2006-12-05,13.092857,13.190000,12.981428,91.269997,13.038571,165709600,DNE
3,2006-12-06,12.948571,13.055715,12.810000,89.830002,12.832857,159546100,Honoring R.W. Apple in Words and Food : About ...
4,2006-12-07,12.861428,12.928572,12.414286,87.040001,12.434286,251206900,"Homebuilders, and Worries Over Jobs, Lead a De..."


In [ ]:
sent_df = apple_data.copy()
sent_df["sentiment_score"] = ''
sent_df["Negative"] = ''
sent_df["Neutral"] = ''
sent_df["Positive"] = ''
sent_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,News,sentiment_score,Negative,Neutral,Positive
0,2006-12-01,13.114285,13.190000,12.871428,91.320000,13.045714,198769900,WHAT'S ON TONIGHT : 8 P.M. (TLC) ASHLEY JUDD A...,,,,
1,2006-12-04,13.125714,13.150000,12.928572,91.120003,13.017143,177384200,More on Housing Prices : The broadest governme...,,,,
2,2006-12-05,13.092857,13.190000,12.981428,91.269997,13.038571,165709600,DNE,,,,
3,2006-12-06,12.948571,13.055715,12.810000,89.830002,12.832857,159546100,Honoring R.W. Apple in Words and Food : About ...,,,,
4,2006-12-07,12.861428,12.928572,12.414286,87.040001,12.434286,251206900,"Homebuilders, and Worries Over Jobs, Lead a De...",,,,


## Get sentiment score

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
%%time
pd.DataFrame.iteritems = pd.DataFrame.items
sentiment_analyzer = SentimentIntensityAnalyzer()
for indx, row in sent_df.T.iteritems():
    try:
        sentence_i = unicodedata.normalize('NFKD', sent_df.loc[indx, 'News'])
        sentence_sentiment = sentiment_analyzer.polarity_scores(sentence_i)
        sent_df.at[indx, 'sentiment_score'] = sentence_sentiment['compound']
        sent_df.at[indx, 'Negative'] = sentence_sentiment['neg']
        sent_df.at[indx, 'Neutral'] = sentence_sentiment['neu']
        sent_df.at[indx, 'Positive'] = sentence_sentiment['pos']
    except TypeError:
        print (sent_df.loc[indx, 'News'])
        print (indx)
        break

CPU times: user 9.18 s, sys: 20.1 ms, total: 9.2 s
Wall time: 9.33 s


In [ ]:
sent_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,News,sentiment_score,Negative,Neutral,Positive
0,2006-12-01,13.114285,13.190000,12.871428,91.320000,13.045714,198769900,WHAT'S ON TONIGHT : 8 P.M. (TLC) ASHLEY JUDD A...,0.7707,0.032,0.905,0.063
1,2006-12-04,13.125714,13.150000,12.928572,91.120003,13.017143,177384200,More on Housing Prices : The broadest governme...,0.872,0.011,0.904,0.085
2,2006-12-05,13.092857,13.190000,12.981428,91.269997,13.038571,165709600,DNE,0.0,0.0,1.0,0.0
3,2006-12-06,12.948571,13.055715,12.810000,89.830002,12.832857,159546100,Honoring R.W. Apple in Words and Food : About ...,0.6858,0.029,0.878,0.093
4,2006-12-07,12.861428,12.928572,12.414286,87.040001,12.434286,251206900,"Homebuilders, and Worries Over Jobs, Lead a De...",-0.6712,0.091,0.869,0.04


In [ ]:
sent_df['Date'] = pd.to_datetime(sent_df['Date'])
sent_df['Date'] = sent_df['Date'].dt.date
sent_df = sent_df.drop(columns=['Negative', 'Positive', 'Neutral', 'News'])
sent_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sentiment_score
0,2006-12-01,13.114285,13.190000,12.871428,91.320000,13.045714,198769900,0.7707
1,2006-12-04,13.125714,13.150000,12.928572,91.120003,13.017143,177384200,0.872
2,2006-12-05,13.092857,13.190000,12.981428,91.269997,13.038571,165709600,0.0
3,2006-12-06,12.948571,13.055715,12.810000,89.830002,12.832857,159546100,0.6858
4,2006-12-07,12.861428,12.928572,12.414286,87.040001,12.434286,251206900,-0.6712


In [ ]:
sent_df.to_csv(rpath + "Apple_Score_2.csv")

In [ ]:
score_df = sent_df.groupby([sent_df['Date']]).mean()
print(score_df.shape)

(2517, 7)


In [ ]:
score_df

,Open,High,Low,Close,Adj Close,Volume,sentiment_score
Date,,,,,,,
2006-12-01,13.114285,13.190000,12.871428,91.320000,13.045714,198769900.0,0.7707
2006-12-04,13.125714,13.150000,12.928572,91.120003,13.017143,177384200.0,0.872
2006-12-05,13.092857,13.190000,12.981428,91.269997,13.038571,165709600.0,0.0
2006-12-06,12.948571,13.055715,12.810000,89.830002,12.832857,159546100.0,0.6858
2006-12-07,12.861428,12.928572,12.414286,87.040001,12.434286,251206900.0,-0.6712
...,...,...,...,...,...,...,...
2016-11-23,111.360001,111.510002,110.330002,111.230003,111.230003,27426400.0,0.0
2016-11-25,111.129997,111.870003,110.949997,111.790001,111.790001,11475900.0,-0.899
2016-11-28,111.430000,112.470001,111.389999,111.570000,111.570000,27194000.0,0.0


## Calculate correlation

First calcualte the covariance matrix

In [ ]:
import numpy as np

def calculate_covariance(sentiment_scores, stock_prices):
    # Calculate the change in stock prices
    stock_changes = np.diff(stock_prices)

    # Check if the price increased or decreased
    price_direction = np.where(stock_changes > 0, 1, 0)

    # Prepend 0 at the beginning of the array
    price_direction = np.insert(price_direction, 0, 0)

    # Calculate the covariance matrix
    covariance_matrix = np.cov(sentiment_scores, price_direction)

    return covariance_matrix

# Example data (replace with your actual data)
sentiment_scores = score_df["sentiment_score"].tolist()
stock_prices = score_df["Adj Close"].tolist()

# Calculate the covariance matrix
covariance_matrix = calculate_covariance(sentiment_scores, stock_prices)

print("Covariance Matrix:")
print(covariance_matrix)


Covariance Matrix:
[[0.39876136 0.00819818]
 [0.00819818 0.24957728]]


Then calculate the correlation coefficient matrix

In [ ]:
def covariance_to_correlation(covariance_matrix):
    # Calculate the standard deviations for each variable
    std_devs = np.sqrt(np.diag(covariance_matrix))

    # Calculate the correlation coefficient matrix
    correlation_matrix = np.zeros_like(covariance_matrix, dtype=float)
    for i in range(len(covariance_matrix)):
        for j in range(len(covariance_matrix)):
            correlation_matrix[i][j] = covariance_matrix[i][j] / (std_devs[i] * std_devs[j])

    return correlation_matrix

# Convert covariance matrix to correlation coefficient matrix
correlation_matrix = covariance_to_correlation(covariance_matrix)

print("Correlation Coefficient Matrix:")
print(correlation_matrix)

Correlation Coefficient Matrix:
[[1.         0.02598713]
 [0.02598713 1.        ]]
